# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need

# 2) instantiate a Spark session 

spark = SparkSession.builder.appName("Data wrangling with Spark SQL").getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# 4) create a view to use with your SQL queries

user_log.createOrReplaceTempView("user_log_table")

# 5) write code to answer the quiz questions 

In [2]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [19]:
spark.sql('''
          SELECT * 
          FROM user_log_table 
          LIMIT 2
          '''
          ).show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

# Question 1

Which page did user id ""(empty string) NOT visit?

In [20]:
# TODO: write your code to answer question 1

spark.sql('''
    SELECT DISTINCT page
    FROM user_log_table
    WHERE page NOT IN
    (
        SELECT DISTINCT page
        FROM user_log_table 
        WHERE userId = ''
    )
    '''
    ).show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

I am more confortable with SQL

# Question 3

How many female users do we have in the data set?

In [21]:
# TODO: write your code to answer question 3

spark.sql('''
    SELECT DISTINCT gender
    FROM user_log_table
    '''
    ).show()

+------+
|gender|
+------+
|     F|
|  null|
|     M|
+------+



In [27]:
spark.sql('''
    SELECT COUNT(DISTINCT userID)
    FROM user_log_table
    WHERE gender = 'F'
    '''
    ).show()

+----------------------+
|count(DISTINCT userID)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [32]:
# TODO: write your code to answer question 4

spark.sql('''
    SELECT artist, COUNT(artist)
    FROM user_log_table
    GROUP BY artist
    ORDER BY COUNT(artist) DESC
    '''
    ).show()

+--------------------+-------------+
|              artist|count(artist)|
+--------------------+-------------+
|            Coldplay|           83|
|       Kings Of Leon|           69|
|Florence + The Ma...|           52|
|            BjÃÂ¶rk|           46|
|       Dwight Yoakam|           45|
|       Justin Bieber|           43|
|      The Black Keys|           40|
|         OneRepublic|           37|
|                Muse|           36|
|        Jack Johnson|           36|
|           Radiohead|           31|
|        Taylor Swift|           29|
|Barry Tuckwell/Ac...|           28|
|          Lily Allen|           28|
|               Train|           28|
|           Daft Punk|           27|
|           Metallica|           27|
|          Nickelback|           27|
|          Kanye West|           26|
|Red Hot Chili Pep...|           24|
+--------------------+-------------+
only showing top 20 rows



In [37]:
spark.sql('''
    SELECT artist, COUNT(artist)
    FROM user_log_table
    GROUP BY artist
    ORDER BY COUNT(artist) DESC
    LIMIT 1
    '''
    ).show()

+--------+-------------+
|  artist|count(artist)|
+--------+-------------+
|Coldplay|           83|
+--------+-------------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [40]:
# TODO: write your code to answer question 5

is_home = spark.sql('''
    SELECT userID
        , page
        , ts
        , CASE
            WHEN page = 'Home' THEN 1 
            ELSE 0 
          END AS is_home
        FROM user_log_table
        WHERE (page = 'NextSong') 
            or (page = 'Home')
    ''')

# keep the results in a new view
is_home.createOrReplaceTempView('is_home_table')

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql('''
    SELECT *
        , SUM(is_home) OVER (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
    FROM is_home_table
    ''')

# keep the results in a view
cumulative_sum.createOrReplaceTempView('period_table')

# find the average count for NextSong
spark.sql('''
    SELECT AVG(count_results)
    FROM
    (
        SELECT COUNT(*) AS count_results
        FROM period_table
        GROUP BY userID, period, page
        HAVING page = 'NextSong'
    ) AS counts
    ''').show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

